In [2]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as BSoup
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install geopy
from geopy.geocoders import Nominatim

!pip install folium # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!pip install geocoder
import geocoder # import geocoder

import seaborn as sns
import matplotlib.pyplot as plt

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
    100% |████████████████████████████████| 102kB 17.3MB/s a 0:00:01
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [3]:
def get_geolocation(address):
  loc = Nominatim().geocode(address)
  geolocation = dict()
  geolocation['latitude'] = loc.latitude
  geolocation['longitude'] = loc.longitude
  return geolocation

In [4]:
toronto_loc = get_geolocation('Toronto, Canada')
toronto_geodata = dict()
toronto_geodata['City'] = 'Toronto'
toronto_geodata['Latitude'] = toronto_loc['latitude']
toronto_geodata['Longitude'] = toronto_loc['longitude']

nyc_loc = get_geolocation('New York City, NY')
nyc_geodata = dict()
nyc_geodata['City'] = 'NYC'
nyc_geodata['Latitude'] = nyc_loc['latitude']
nyc_geodata['Longitude'] = nyc_loc['longitude']


In [5]:

# @hidden cell
CLIENT_ID = 'KRBOQIRZJFXTS2BGSK2CN0YYB2CGYGFA1OUN1HMDEKSZUMPP' 
CLIENT_SECRET = 'C1V3DPD45QVRMHUKMUGBYHSRTJE4S525EVT0AUAXC3GB40YO' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')

Your credentails:


In [9]:
def getNearbyVenues(names, lat, lng, radius=500):

  venues_list=[]
#   for name, lat, lng in zip(names, latitudes, longitudes):
  name = names.split(',')[0]
#   print(name)
  LIMIT = 100

  # create the API request URL
  url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
      CLIENT_ID, 
      CLIENT_SECRET, 
      VERSION, 
      lat, 
      lng, 
      radius, 
      LIMIT)

  # make the GET request
  results = requests.get(url).json()["response"]['groups'][0]['items']
#   print(results)

  # return only relevant information for each nearby venue
  venues_list.append([(

      v['venue']['name'], 
      str(v['venue']['location']['lat']), 
      str(v['venue']['location']['lng']),  
      v['venue']['categories'][0]['name']) for v in results])

  nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
  nearby_venues.columns = [
                'Venue', 
                'Latitude', 
                'Longitude', 
                'Venue Category']

  return(nearby_venues)

In [10]:
nyc_venues = getNearbyVenues(names=nyc_geodata['City'], lat=nyc_geodata['Latitude'], lng=nyc_geodata['Longitude'])
toronto_venues = getNearbyVenues(names=toronto_geodata['City'], lat=toronto_geodata['Latitude'], lng=toronto_geodata['Longitude'])

 1. Grouping the above data on the basis of Venue Categories

In [11]:
nyc_geoloc_groups = (nyc_venues.groupby('Venue Category').apply(lambda x: [i for i in map(lambda p, q: (p,q), x.Latitude, x.Longitude)]).reset_index())
nyc_geoloc_groups.rename(columns={0: 'Coordinates_NYC'}, inplace=True)
print(nyc_geoloc_groups.head(3))

toronto_geoloc_groups = (toronto_venues.groupby('Venue Category').apply(lambda y: [i for i in map(lambda p, q: (p,q), y.Latitude, y.Longitude)]).reset_index())
toronto_geoloc_groups.rename(columns={0: 'Coordinates_Toronto'}, inplace=True)
print(toronto_geoloc_groups.head(3))

        Venue Category                                    Coordinates_NYC
0  American Restaurant  [(40.7114909233789, -74.00683195829815), (40.7...
1         Antique Shop          [(40.71528423132827, -74.00878093952018)]
2          Art Gallery         [(40.714294907023415, -74.00827965625265)]
        Venue Category                                Coordinates_Toronto
0  American Restaurant  [(43.6505120350372, -79.38813081159722), (43.6...
1          Art Gallery  [(43.654002860337386, -79.39292172707437), (43...
2           Art Museum          [(43.65439630500274, -79.38650010906946)]


2. Merging the geolocation data of NYC for each row of the grouped data¶


In [12]:
nyc_venue_catg_grouped = nyc_venues.groupby('Venue Category').count().reset_index()
nyc_venue_groups = (nyc_venues.groupby('Venue Category').apply(lambda x: ','.join(x.Venue)).reset_index())
nyc_venue_grouped = (nyc_venue_catg_grouped.sort_values('Venue', ascending=False))[['Venue Category', 'Venue']]

nyc_venue_grouped.rename(columns={'Venue': 'Count_NYC'}, inplace=True)
nyc_venue_groups.rename(columns={0: 'Venues_NYC'}, inplace=True)

nyc_count_venues = pd.merge(nyc_venue_groups, nyc_venue_grouped, on="Venue Category", how="inner")
# print(nyc_merged_venues.sort_values('Count_NYC', ascending=False).head(5))

nyc_count_data = pd.merge(nyc_count_venues, nyc_geoloc_groups, on="Venue Category", how="inner")
nyc_count_data
# print(list(nyc_venue_groups.loc[5]))

,Venue Category,Venues_NYC,Count_NYC,Coordinates_NYC
0,American Restaurant,"Temple Court,Little Park",2,"[(40.7114909233789, -74.00683195829815), (40.7..."
1,Antique Shop,Philip Williams Posters,1,"[(40.71528423132827, -74.00878093952018)]"
2,Art Gallery,One Art Space,1,"[(40.714294907023415, -74.00827965625265)]"
3,Baby Store,Babesta,1,"[(40.71476, -74.00928)]"
4,Bakery,"Takahachi Bakery,Woops!,Arcade Bakery",3,"[(40.713652845301894, -74.0088038953017), (40...."
5,Bar,Maxwell's Bar & Restaurant,1,"[(40.71494392643706, -74.00648461952706)]"
6,Bookstore,The Mysterious Bookshop,1,"[(40.71490950153982, -74.00948027011903)]"
7,Boxing Gym,Church Street Boxing Gym,1,"[(40.713353574017404, -74.00906676030888)]"
8,Building,Woolworth Building,1,"[(40.712559078189095, -74.00796435178208)]"
9,Burger Joint,"Shake Shack,Schnipper's Quality Kitchen",2,"[(40.710528, -74.008988), (40.7137184634891, -..."


3. Merging the geolocation data of Toronto for each row of the grouped data¶


In [14]:
toronto_venue_catg_grouped = toronto_venues.groupby('Venue Category').count().reset_index()
toronto_venue_groups = (toronto_venues.groupby('Venue Category').apply(lambda x: ','.join(x.Venue)).reset_index())
toronto_venue_grouped = (toronto_venue_catg_grouped.sort_values('Venue', ascending=False))[['Venue Category', 'Venue']]

toronto_venue_grouped.rename(columns={'Venue': 'Count_Toronto'}, inplace=True)
toronto_venue_groups.rename(columns={0: 'Venues_Toronto'}, inplace=True)

toronto_count_venues = pd.merge(toronto_venue_groups, toronto_venue_grouped, on="Venue Category", how="inner")
#print((toronto_venue_groups.loc[27])[1].split(','))

toronto_count_data = pd.merge(toronto_count_venues, toronto_geoloc_groups, on="Venue Category", how="inner")
toronto_count_data
#print(toronto_merged_venues.sort_values('Count_Toronto', ascending=False).head(5))

,Venue Category,Venues_Toronto,Count_Toronto,Coordinates_Toronto
0,American Restaurant,"Nota Bene,JOEY",2,"[(43.6505120350372, -79.38813081159722), (43.6..."
1,Art Gallery,"Art Gallery of Ontario,Norma Ridley Members’ L...",4,"[(43.654002860337386, -79.39292172707437), (43..."
2,Art Museum,Textile Museum of Canada,1,"[(43.65439630500274, -79.38650010906946)]"
3,Arts & Crafts Store,"Aboveground Art Supplies,Curry's Artist Supply",2,"[(43.652645648070006, -79.3909251764317), (43...."
4,Asian Restaurant,Noodle King,1,"[(43.65170632794881, -79.38304628912547)]"
5,Bakery,Butter Baker,1,"[(43.65566927096165, -79.3842978108375)]"
6,Bar,"Sin and Redemption,Queen St. Warehouse",2,"[(43.65466135819523, -79.39173728227614), (43...."
7,Breakfast Spot,"Karine's,Eggspectation Bell Trinity Square,Hil...",3,"[(43.653698928318526, -79.39074313568692), (43..."
8,Bubble Tea Shop,"Chatime 日出茶太,Bubble Republic Tea House 奶茶黨部",2,"[(43.65554164147378, -79.38468427043244), (43...."
9,Café,"Cafe Plenty,Jimmy's Coffee,Red Eye Espresso,AG...",4,"[(43.65457125894357, -79.38944976536852), (43...."


4. Combining the geolocation data of NYC and Toronto for each row of the grouped data¶


In [15]:
combined_count_data = pd.merge(nyc_count_data, toronto_count_data, on="Venue Category", how="inner")
combined_count_data

,Venue Category,Venues_NYC,Count_NYC,Coordinates_NYC,Venues_Toronto,Count_Toronto,Coordinates_Toronto
0,American Restaurant,"Temple Court,Little Park",2,"[(40.7114909233789, -74.00683195829815), (40.7...","Nota Bene,JOEY",2,"[(43.6505120350372, -79.38813081159722), (43.6..."
1,Art Gallery,One Art Space,1,"[(40.714294907023415, -74.00827965625265)]","Art Gallery of Ontario,Norma Ridley Members’ L...",4,"[(43.654002860337386, -79.39292172707437), (43..."
2,Bakery,"Takahachi Bakery,Woops!,Arcade Bakery",3,"[(40.713652845301894, -74.0088038953017), (40....",Butter Baker,1,"[(43.65566927096165, -79.3842978108375)]"
3,Bar,Maxwell's Bar & Restaurant,1,"[(40.71494392643706, -74.00648461952706)]","Sin and Redemption,Queen St. Warehouse",2,"[(43.65466135819523, -79.39173728227614), (43...."
4,Café,"Konditori,Pisillo Italian Cafe,Mulberry & Vine...",4,"[(40.70947390929377, -74.00663033243585), (40....","Cafe Plenty,Jimmy's Coffee,Red Eye Espresso,AG...",4,"[(43.65457125894357, -79.38944976536852), (43...."
5,Coffee Shop,"The Wooly Daily,Birch Coffee,Blue Spoon Coffee...",6,"[(40.71213724343698, -74.00839457685005), (40....","The Library Specialty Coffee,M Square Coffee C...",6,"[(43.65441282740799, -79.39090161351724), (43...."
6,Cosmetics Shop,"e.l.f. Studio,VIP Nail",2,"[(40.71040890541702, -74.00793520565746), (40....",Essential Beauty Bar,1,"[(43.65051268683632, -79.38802119167447)]"
7,Dance Studio,"Gibney Dance Center Downtown,Downtown Dance Fa...",2,"[(40.71392316324669, -74.0056614920451), (40.7...",The Underground Dance Centre,1,"[(43.64974018262459, -79.38804462470118)]"
8,French Restaurant,"Augustine,Racines",2,"[(40.71130993467075, -74.00666043815025), (40....",Midi Bistro,1,"[(43.65587131386703, -79.39209126452964)]"
9,Greek Restaurant,GRK Fresh Greek - Financial District,1,"[(40.7097995488229, -74.0070113011889)]",Estiatorio Volos,1,"[(43.65032912559998, -79.38453301297716)]"


5. Performing Clustering on the aforeformed data using k-Means Clustering algorithm¶


In [16]:
k_cluster_num = 5
columns = ['NYC_Venue_Count', 'Toronto_Venue_Count']
data_temp = {'NYC_Venue_Count': combined_count_data['Count_NYC'], 'Toronto_Venue_Count': combined_count_data['Count_Toronto']}
# print(toronto_venue_grouped['Venue'])
data_temp_df = pd.DataFrame(data_temp, columns=columns)
print(data_temp_df)
k_means_count = KMeans(n_clusters=k_cluster_num, random_state=0).fit(data_temp_df)
print(k_means_count.labels_)

    NYC_Venue_Count  Toronto_Venue_Count
0                 2                    2
1                 1                    4
2                 3                    1
3                 1                    2
4                 4                    4
5                 6                    6
6                 2                    1
7                 2                    1
8                 2                    1
9                 1                    1
10                3                    1
11                3                    1
12                1                    1
13                1                    4
14                1                    1
15                1                    1
16                1                    1
17                3                    1
18                2                    1
19                1                    1
20                1                    1
21                1                    3
22                1                    1
[0 1 0 2 4 3 0 0

6. Merging the Cluster labels with the Count data¶


In [17]:
# Creating a DataFrame with both Cluster-values and top-10 venues of each neighborhood
count_combined = combined_count_data
count_combined['Cluster Labels'] = k_means_count.labels_
print(count_combined.head(3))
# count_combined = count_combined.join(neighbourhoods_sorted.set_index('Neighborhood'), on='Neighborhood')

        Venue Category                             Venues_NYC  Count_NYC  \
0  American Restaurant               Temple Court,Little Park          2   
1          Art Gallery                          One Art Space          1   
2               Bakery  Takahachi Bakery,Woops!,Arcade Bakery          3   

                                     Coordinates_NYC  \
0  [(40.7114909233789, -74.00683195829815), (40.7...   
1         [(40.714294907023415, -74.00827965625265)]   
2  [(40.713652845301894, -74.0088038953017), (40....   

                                      Venues_Toronto  Count_Toronto  \
0                                     Nota Bene,JOEY              2   
1  Art Gallery of Ontario,Norma Ridley Members’ L...              4   
2                                       Butter Baker              1   

                                 Coordinates_Toronto  Cluster Labels  
0  [(43.6505120350372, -79.38813081159722), (43.6...               0  
1  [(43.654002860337386, -79.39292172707437

7. Grouping and combining each returned venue and location of NYC


In [18]:
# onehot-encoding the venues-data:
nyc_venues_onehot = pd.get_dummies(nyc_venues[['Venue Category']], prefix='', prefix_sep='')
# print(nyc_venues_onehot)

nyc_venues_onehot['Venue Category'] = nyc_venues['Venue Category']
# nyc_venues_onehot
nyc_venues_grouped_mean = nyc_venues_onehot.groupby('Venue Category').mean().reset_index()

nyc_mean_data = pd.merge(nyc_venues_grouped_mean, nyc_geoloc_groups, on="Venue Category", how="inner")

nyc_venues_mean_temp = nyc_venues_grouped_mean.drop('Venue Category', 1)
k_means_avg_nyc = KMeans(n_clusters=k_cluster_num, random_state=0).fit(nyc_venues_mean_temp)
nyc_mean_merged = nyc_venues_grouped_mean
nyc_mean_merged['Cluster Labels'] = k_means_avg_nyc.labels_
nyc_mean_merged = pd.merge(nyc_mean_merged[['Venue Category', 'Cluster Labels']], nyc_venues_onehot, on='Venue Category', how='inner')
nyc_mean_merged = pd.merge(nyc_mean_merged, nyc_venues[['Venue Category', 'Venue', 'Latitude', 'Longitude']], on='Venue Category', how='inner')
nyc_mean_merged.drop_duplicates(inplace=True)

8.Grouping and combining each returned venue and location of Toronto


In [19]:
# onehot-encoding the venues-data:
toronto_venues_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix='', prefix_sep='')
# print(toronto_venues_onehot)

toronto_venues_onehot['Venue Category'] = toronto_venues['Venue Category']
toronto_venues_grouped_mean = toronto_venues_onehot.groupby('Venue Category').mean().reset_index()

toronto_mean_data = pd.merge(toronto_venues_grouped_mean, toronto_geoloc_groups, on="Venue Category", how="inner")

toronto_venues_mean_temp = toronto_venues_grouped_mean.drop('Venue Category', 1)
k_means_avg_toronto = KMeans(n_clusters=k_cluster_num, random_state=0).fit(toronto_venues_mean_temp)
toronto_mean_merged = toronto_venues_grouped_mean
toronto_mean_merged['Cluster Labels'] = k_means_avg_toronto.labels_
toronto_mean_merged = pd.merge(toronto_mean_merged[['Venue Category', 'Cluster Labels']], toronto_venues_onehot, on='Venue Category', how='inner')
# toronto_mean_merged[['Venue Category', 'Cluster Labels']]
toronto_mean_merged = pd.merge(toronto_mean_merged, toronto_venues[['Venue Category', 'Venue', 'Latitude', 'Longitude']], on='Venue Category', how='inner')
toronto_mean_merged.drop_duplicates(inplace=True)

9. Visualizing all the returned venues of NYC and Toronto¶


Creating a function to visualize all the venues of NYC and Toronto using Folium Maps¶


In [20]:
def create_city_map(merged_data, geoloc_data, common_data):
  base_lat = geoloc_data['Latitude']
  base_lng = geoloc_data['Longitude']
  map_clusters = folium.Map(location=[base_lat, base_lng], zoom_start=16)

  # set color scheme for the clusters
  x = np.arange(k_cluster_num)
  ys = [i+x+(i*x)**2 for i in range(k_cluster_num)]
  colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
  rainbow = [colors.rgb2hex(i) for i in colors_array]

  # add markers to the map
  markers_colors = []
  for lat, lon, poi, cluster, categ in zip(merged_data['Latitude'], merged_data['Longitude'], merged_data['Venue'], merged_data['Cluster Labels'], merged_data['Venue Category']):
    label = folium.Popup('{} ({}) - Cluster {}'.format(str(poi), categ, str(cluster)), parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lon)],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

  return map_clusters

Map of Popular Venues in NewYork City¶


In [21]:
nyc_overall_map = create_city_map(nyc_mean_merged, nyc_geodata, count_combined)
nyc_overall_map

Map of Popular Venues in Toronto

In [22]:
toronto_overall_map = create_city_map(toronto_mean_merged, toronto_geodata, count_combined)
toronto_overall_map

10. Creating a function to display each venue from the combined count data using Folium Map


In [23]:
""" 
Creating map of Cities's Neighborhood using Folium
"""

# defining the Folium Map in a function
def create_map(city_data, geoloc_data):
  base_lat = float(geoloc_data['Latitude'])
  base_lng = float(geoloc_data['Longitude'])
  map_clusters = folium.Map(location=[base_lat, base_lng], zoom_start=16)

  # set color scheme for the clusters

  rainbow = ['#F84583', '#4868F7', '#53DEE8', '#00F950', '#F9FA20']
  geodata_param = city_data['Coordinates_{}'.format(geoloc_data['City'])]
  venue_param = city_data['Venues_{}'.format(geoloc_data['City'])]

  # add markers to the map
  markers_colors = []

  for geo_data, venues, cluster, categ in zip(geodata_param, venue_param, city_data['Cluster Labels'], city_data['Venue Category']):
#     print('{}: [{}] - cluster {}'.format(venues, geo_data, cluster))
    for geoloc, poi in zip(geo_data, venues.split(',')):
#       print('{1}: {0}'.format(geoloc, poi))
      label = folium.Popup(str(poi) + ' ( {} ) - Cluster {}'.format(categ, str(cluster)), parse_html=True)
      folium.CircleMarker(
          [float(geoloc[0]), float(geoloc[1])],
          radius=5,
          popup=label,
          color=rainbow[cluster-1],
          fill=True,
          fill_color=rainbow[cluster-1],
          fill_opacity=1).add_to(map_clusters)
  return map_clusters

Displaying New York City Venues¶


In [24]:
nyc_cluster_map = create_map(count_combined, nyc_geodata)
nyc_cluster_map

Cluster Analysis for NYC

1

In [26]:
first_cluster = count_combined.loc[count_combined['Cluster Labels'] == 0, count_combined.columns[[0] + list(range(1, count_combined.shape[1]))]]
print(first_cluster.shape[0])
nyc_first_cluster = create_map(first_cluster, nyc_geodata)
nyc_first_cluster

9


second

In [27]:
second_cluster = count_combined.loc[count_combined['Cluster Labels'] == 1, count_combined.columns[[0] + list(range(1, count_combined.shape[1]))]]
print(len(second_cluster))
nyc_second_cluster = create_map(second_cluster, nyc_geodata)
nyc_second_cluster

3


third

In [28]:

third_cluster = count_combined.loc[count_combined['Cluster Labels'] == 2, count_combined.columns[[0] + list(range(1, count_combined.shape[1]))]]
print(len(third_cluster))
nyc_third_cluster = create_map(third_cluster, nyc_geodata)
nyc_third_cluster

9


Displaying Toronto Venues

In [29]:
toronto_cluster_map = create_map(count_combined, toronto_geodata)
toronto_cluster_map

Analysing Clusters of Toronto

First cluster

In [30]:
first_cluster = count_combined.loc[count_combined['Cluster Labels'] == 0, count_combined.columns[[0] + list(range(1, count_combined.shape[1]))]]
print(first_cluster.shape[0])
toronto_first_cluster = create_map(first_cluster, toronto_geodata)
toronto_first_cluster

9


second

In [32]:
second_cluster = count_combined.loc[count_combined['Cluster Labels'] == 1, count_combined.columns[[0] + list(range(1, count_combined.shape[1]))]]
print(second_cluster.shape[0])
toronto_second_cluster = create_map(second_cluster, toronto_geodata)
toronto_second_cluster

3
